In [2]:
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
import time

In [1]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_dataset = mnist.train.images
train_labels = mnist.train.labels

valid_dataset = mnist.validation.images
valid_labels = mnist.validation.labels

test_dataset = mnist.test.images
test_labels = mnist.test.labels

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

NameError: name 'input_data' is not defined

In [5]:
image_size = 28
num_channels = 1

def reformat(data):
    return np.reshape(data, (-1, image_size, image_size, num_channels))

train_dataset = reformat(train_dataset)
valid_dataset = reformat(valid_dataset)
test_dataset = reformat(test_dataset)

print('training set:', train_dataset.shape, train_labels.shape)
print('validation set:', valid_dataset.shape, valid_labels.shape)
print('test set:', test_dataset.shape, test_labels.shape)

training set: (55000, 28, 28, 1) (55000, 10)
validation set: (5000, 28, 28, 1) (5000, 10)
test set: (10000, 28, 28, 1) (10000, 10)


In [58]:
batch_size = 100
filter_size = 5
depth1 = 6
depth2 = 16
fc_input_size = 400 # tentar calcular com o shape quando tiver tempo
fc1_size = 120
fc2_size = 84
num_labels = 10

graph = tf.Graph()

with graph.as_default():
    
    # inputs
    data = tf.placeholder(tf.float32, shape=(None, image_size, image_size, num_channels))
    labels = tf.placeholder(tf.float32, shape=(None, num_labels))
    
    # convolutional layers learnable parameters
    layer1_weights = tf.Variable(tf.truncated_normal([filter_size, filter_size, num_channels, depth1], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth1]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([filter_size, filter_size, depth1, depth2], stddev=0.1))
    layer2_biases = tf.Variable(tf.zeros([depth2]))
    
    # feedforward layers learnable parameters
    layer3_weights = tf.Variable(tf.truncated_normal([fc_input_size, fc1_size], stddev=0.1))
    layer3_biases = tf.Variable(tf.zeros([fc1_size]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([fc1_size, fc2_size], stddev=0.1))
    layer4_biases = tf.Variable(tf.zeros([fc2_size]))
    
    layer5_weights = tf.Variable(tf.truncated_normal([fc2_size, num_labels], stddev=0.1))
    layer5_biases = tf.Variable(tf.zeros([num_labels]))
    
    # build the model
    def model(data):
        conv1 = tf.nn.relu(tf.add(tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME'), layer1_biases))
        pool1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        conv2 = tf.nn.relu(tf.add(tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='VALID'), layer2_biases))
        pool2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
        
        shape = pool2.get_shape().as_list()
        reshaped = tf.reshape(pool2, [-1, 400])
        fc1 = tf.nn.relu(tf.add(tf.matmul(reshaped, layer3_weights), layer3_biases))
        
        fc2 = tf.nn.relu(tf.add(tf.matmul(fc1, layer4_weights), layer4_biases))
        
        return tf.add(tf.matmul(fc2, layer5_weights), layer5_biases)
    
    
    # optimizing
    logits = model(data)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
    optmizer = tf.train.AdamOptimizer(0.001).minimize(loss)
    
    # prediction and accuracy
    prediction = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
    model_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [64]:
num_epochs = 10 # if taking too long to run, changing this to 10 should make little to no difference
num_batches = int(mnist.train.num_examples/batch_size)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for epoch in range(num_epochs):
        print('epoch', epoch + 1)
        start_time = time.time()
        for batch in range(num_batches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            batch_x = reformat(batch_x)
            x_valid = reformat(mnist.validation.images)
            y_valid = mnist.validation.labels
            _ = session.run(optmizer, feed_dict={data:batch_x, labels:batch_y})
            l = session.run(loss, feed_dict={data:batch_x, labels:batch_y})
            valid_acc = session.run(model_accuracy, feed_dict={data:x_valid, labels:y_valid})
        print('loss:', l)
        print('validation accuracy:', valid_acc)
        print('--- %s minutes ---' % ((time.time() - start_time)/60))
        print()
    
    test_data = reformat(mnist.test.images)
    test_acc = session.run(model_accuracy, feed_dict={data:test_data, labels:mnist.test.labels})
    print('test accuracy:', test_acc)
    print('--- %s minutes ---' % ((time.time() - start_time)/60))

epoch 0
loss: 2.27535
validation accuracy: 0.1682
--- 12.5494204322 minutes ---

epoch 1


KeyboardInterrupt: 